### setup & load delta tables

Quering parquet files

In [0]:
%sql
select * from parquet.`/Volumes/exercise/exercise/exercise/flights-1m.parquet`;

CTAS

In [0]:
%sql
USE CATALOG exercise;

CREATE OR REPLACE TABLE exercise.FLIGHTS_DATA
USING DELTA AS
select * from parquet.`/Volumes/exercise/exercise/exercise/flights-1m.parquet`;

DESCRIBE exercise.FLIGHTS_DATA;

In [0]:
%sql 
DESCRIBE EXTENDED exercise.FLIGHTS_DATA;

CATALOG, SCHEMA & TABLES IN DATABRICKS

In [0]:
%sql 
DESCRIBE CATALOG `exercise`;

LOAD INCREMENTALLY

In [0]:
%sql
USE CATALOG exercise;

DROP TABLE IF EXISTS exercise.FLIGHTS_DATA;

CREATE TABLE exercise.FLIGHTS_DATA;

COPY INTO exercise.FLIGHTS_DATA
FROM '/Volumes/exercise/exercise/exercise/flights-1m.parquet'
FILEFORMAT=parquet
COPY_OPTIONS('mergeSchema' = 'true');

Quering CSV files

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW sample_csv_data
AS
select * from csv.`/Volumes/exercise/exercise/exercise/sample4.csv`;

select * from sample_csv_data limit 5;

### BASIC TRANSFORMATIONS

cloning delta live tables

In [0]:
%sql
CREATE OR REPLACE TABLE deep_cloned_FLIGHTS_DATA DEEP CLONE exercise.FLIGHTS_DATA;

In [0]:
%sql
CREATE OR REPLACE TABLE shallow_cloned_FLIGHTS_DATA SHALLOW CLONE exercise.FLIGHTS_DATA;

MERGE UPDATES

In [0]:
%sql
-- target table
CREATE OR REPLACE TABLE Products(
   ProductID INT,
   ProductName VARCHAR(100),
   Rate DECIMAL(10, 2)
);

INSERT INTO Products VALUES
(1, 'Tea', 10.00),
(2, 'Coffee', 20.00),
(3, 'Muffin', 30.00),
(4, 'Biscuit', 40.00);

-- source table
CREATE OR REPLACE TABLE UpdatedProducts(
   ProductID INT,
   ProductName VARCHAR(100),
   Rate DECIMAL(10, 2)
);

INSERT INTO UpdatedProducts
VALUES
(1, 'Tea', 10.00),     
(2, 'Coffee', 25.00),   
(3, 'Muffin', 35.00),   
(5, 'Pizza', 60.00);    

In [0]:
%sql
MERGE INTO Products AS TARGET
USING UpdatedProducts AS SOURCE
ON TARGET.ProductID = SOURCE.ProductID
WHEN MATCHED AND (TARGET.ProductName <> SOURCE.ProductName OR TARGET.Rate <> SOURCE.Rate)
THEN 
   UPDATE SET TARGET.ProductName = SOURCE.ProductName, TARGET.Rate = SOURCE.Rate
WHEN NOT MATCHED BY TARGET
THEN 
   INSERT (ProductID, ProductName, Rate) 
   VALUES (SOURCE.ProductID, SOURCE.ProductName, SOURCE.Rate)
WHEN NOT MATCHED BY SOURCE
THEN 
   DELETE;

UDF

In [0]:
import datetime

def get_day_of_week(date_string):
    date_object = datetime.datetime.strptime(date_string, '%Y-%m-%d')
    return date_object.strftime('%A')
  
print(get_day_of_week('2025-05-06'))

In [0]:
%sql
CREATE OR REPLACE FUNCTION exercise.get_day_of_week(date_string STRING)
RETURNS STRING
RETURN date_format(date_string, 'EEEE');

In [0]:
get_day_of_week('2025-05-06')

In [0]:
%sql
select *, exercise.get_day_of_week(FL_DATE) from parquet.`/Volumes/exercise/exercise/exercise/flights-1m.parquet`;